In [ ]:
!pip install siibra==0.4a29

In [ ]:
import siibra
assert siibra.__version__ >= "0.4a28"
from nilearn import plotting
import matplotlib.pyplot as plt
%matplotlib notebook

### Define two points in MNI space

We start by specifying some points in the reference space. Here, we use the MNI space. We can find such points, amongst other possibilities, by clicking them in the [siibra-explorer](https://siibra-explorer.apps.hbp.eu/). `siibra` has specific data types for points and point sets, which are aware of the reference space used, and can be warped between reference spaces on demand.

In [ ]:
# These are copy-pasted from the interactive atlas viewer:
points = siibra.PointSet(
    [
        "-25.650mm, -2.750mm, -33.750mm",
        "-37.350mm, -81.050mm, -6.300mm"
    ], 
    space='mni152', sigma_mm=5)

In [ ]:
plotting.view_markers(list(map(tuple,points)), ['red', 'cyan'], marker_size=10)  

### Assign brain regions to the 3D points

We assign the points to brain regions from the Julich-Brain cytoarchitectonic atlas, using a certain location tolerance of 5mm standard deviation.

In [ ]:
julich_pmaps = siibra.get_map(
    parcellation=siibra.parcellations.JULICH_BRAIN_PROBABILISTIC_CYTOARCHITECTONIC_MAPS_V2_9,
    space=siibra.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC,
    maptype=siibra.MapType.STATISTICAL
)
assignments = julich_pmaps.assign(points)

The result of the assignment is a pandas dataframe, where each row describes a region assigned to one of the input structures (which in this case are the provided two points, 0 and 1). Each assignment has some scores describing the relation of the point, represented as a 3D Gaussian kernel with bandwidth corresponding to the point uncertainty, and the respectivty assigned statistical map from the parcellation:

- "Value" refers to the average value of the statistcal map under the point kernel
- "Correlation" is Pearson's correlation coefficient between the statistical map and the point kernel
- "IoU" is the intersection over union between both
- "Contains" is the intersection over area of the statistical map (indicating containedness of the point)
- "Contained" is the intersection over area of the point (indicating containedness of the statistical map)

To see the strongest assignments, we sort by containedness and filter the table:

In [ ]:
assignments.query("Value > 0.9 & Contains > 0.6")

Let's look at the areas with highest containedness assigned to each point.

In [ ]:
assignments.sort_values(by='Contains', ascending=False, inplace=True)

a1 = assignments.query('Structure==0').iloc[0]
a2 = assignments.query('Structure==1').iloc[0]

for a in [a1, a2]:
    
    # fetch the probability map corresponding to the assigned regions
    statmap = julich_pmaps.fetch(region=a.Region)

    # plot the map, center at the point
    view = plotting.plot_stat_map(
        statmap, 
        title=f"{a.Region} ({a.Contains:.2f})", 
        cmap='viridis', 
        cut_coords=a.Centroid)

    # add the point
    view.add_markers([a.Centroid])

### Look for white matter bundles connecting the regions

Next, we look at the most probable region associated to each point, considering them as a source and target region to investigate connectivity. In order to find white matter fibre bundles which are likely to connect them, we use the probability maps of the white matter fibre bundle parcellation, and assign both the source and target region from Julich-Brain to the fibre bundles. The intersection of the resulting bundles gives us those that are likely to provide connections from source to target.

In [ ]:
# get the probabilistic maps of long fibre bundles
bundlemaps = siibra.get_map(
    siibra.parcellations.PROBABILISTIC_LONG_WHITE_MATTER_BUNDLE_MAPS,
    siibra.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC,
    siibra.MapType.STATISTICAL
)

# assign the source and target region to the bundles, 
# using their probability maps found above.
bundles1 = bundlemaps.assign(julich_pmaps.fetch(region=a1.Region))
bundles2 = bundlemaps.assign(julich_pmaps.fetch(region=a2.Region))
                
# intersect the two sets of related fiber bundles
# we use panda's merge functionality for this operation.
intersection = bundles1.merge(right=bundles2, how='inner', on=['Region'])
intersection

Now we plot the bundles in the intersection.

In [ ]:
for i, row in intersection.iterrows():
    view = plotting.plot_glass_brain(
        bundlemaps.fetch(region=row.Region),
        title=f"{a1.Region} and {a2.Region} connected by {row.Region} ")
    
    view.add_overlay(julich_pmaps.fetch(region=a1.Region))
    view.add_overlay(julich_pmaps.fetch(region=a2.Region))

    view.add_markers(points.as_list())

### Find connectivity in terms of streamlines from DTI

We also investigate the connectivtiy of the two regions as measured by in-vivo imaging. To do so, we select the first region in the atlas, and search for connectivity profiles.

In [ ]:
# get structural connectivity matrices for  Julich Brain
features = siibra.features.get(
    julich_pmaps.parcellation,
    siibra.features.connectivity.StreamlineCounts
)

# get the averaged connectivity across subjects from the first feature
sc = features[0].get_matrix()

# get the connectivity profile with the strongest 10 connections for the first matched region
profile = features[0].get_profile(a1.Region, max_rows=10)

profile.plot()